In [1]:
import pandas as pd
import json
import os
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
load_dotenv("template.env")

True

In [2]:
# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError("La clave de API no está configurada en el archivo .env")
    
client = OpenAI() #creando un objeto de la clase

In [1]:
dataset_folder = os.getenv("DATASET_FOLDER")
dataset_path = str(dataset_folder) + "Alonso_2014_SpanishAoA.xlsx"

df = pd.read_excel(dataset_path)
#df = df.sample(5)
df.head()

NameError: name 'os' is not defined

In [4]:
#FUNCTION DECLARATION

def extract_data(new_line):
	res = new_line["response"]["body"]["choices"][0]["message"]["content"]
	res = json.loads(res)
	return res

def create_file_from_tasks(tasks,file_name):
	with open(file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')


def create_batch(file_name):
	batch_file = client.files.create(
		file = open(file_name, "rb"),
		purpose = "batch"
	)
	batch_job = client.batches.create(
		input_file_id = batch_file.id,
		endpoint = "/v1/chat/completions",
		completion_window = "24h"
	)
	return batch_job

def get_line_file(file_name,line,extract_func):
	with open(file_name, 'r') as f:
		for line_number, theline in enumerate(f):
			if line_number == line:
				res = theline
				break
	res = json.loads(res)
	return extract_func(res)

def generate_task(index,prompt,desc):
	task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Esto es lo que tendrías en tu llamada a la API de Chat Completions
            "model": "gpt-4o-mini",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": desc
                }
            ],
        }
    }
	return task

def create_task_from_json(json_object,index,prompt):
	desc = json.dumps({"palabra":json_object["word"]})
	task = generate_task(
		index, prompt,desc,
	)
	return task

def create_task_array_from_dataframe(df,prompt):
	tasks = []
	for index, row in df.iterrows():
		task = create_task_from_json(row,index,prompt)
		tasks.append(task)
	return tasks


In [5]:
#PROMPTS

#AGE PROMPT
categorize_system_prompt_paraphrase ='''
La edad de adquisición (AoA) de una palabra se refiere a la edad en la que se aprendió una palabra por primera vez. 
En concreto, cuándo una persona habría entendido por primera vez esa palabra si alguien la hubiera utilizado delante de ella, incluso cuando aún no la hubiera dicho, leído o escrito. 
Calcule la edad media de adquisición (AoA) de la palabra {palabra} para un hablante nativo de español.

El formato de salida debe ser un objeto JSON: {AoA: número //AoA de la palabra expresado en años, puede incluir decimales, Word: palabra //string}
'''

In [6]:
#Set output folder
output_folder = os.getenv("OUTPUT_FOLDER")
def out_file(file_name): return (str(output_folder) + file_name)

In [7]:
#AGE TASK
tasks_array = [create_task_array_from_dataframe(df,categorize_system_prompt_paraphrase)]
file_array = [out_file("batch_job_mmlu_aoa_alonso.jsonl")]

In [8]:
#GENERATE TASK FILES
for i in range(0,len(tasks_array)):
	create_file_from_tasks(tasks_array[i],file_array[i])

In [10]:
#GENERATE BATCH
batch_jobs = []
for i in range(0,len(tasks_array)):
	ba_jo= create_batch(file_array[i])
	batch_jobs.append(ba_jo)

In [14]:
#COMPLETION_CHECK
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	print(batch)
	result_file_id = batch.output_file_id
	print(batch.status)

Batch(id='batch_67c61111cc0881909b420a931a884255', completion_window='24h', created_at=1741033745, endpoint='/v1/chat/completions', input_file_id='file-82KjZr2t4jwG55nfei7qxT', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1741052458, error_file_id=None, errors=None, expired_at=None, expires_at=1741120145, failed_at=None, finalizing_at=1741051672, in_progress_at=1741033747, metadata=None, output_file_id='file-BKGHS5UtgLCfL7MZhFpfSc', request_counts=BatchRequestCounts(completed=7039, failed=0, total=7039))
completed


In [15]:
#OUTPUT FILES GENERATOR
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	result_file_id = batch.output_file_id

	result = client.files.content(result_file_id).content

	result_file_name = file_array[i].replace(".json","_result.json")

	with open(result_file_name, 'wb') as file:
		file.write(result)

In [17]:
import re

def clean_text(text):
    if isinstance(text, str):
        # Eliminar caracteres de control ASCII (excepto saltos de línea/tabulación)
        text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]", "", text)
        return text.strip()
    return text

#Cleaning function
def clean(file_array):
	rows = []
	errors = []
	acum = 0

	f_a = file_array[i].replace(".json","_result.json")
	with open(f_a, 'r') as f:
		for line in f:
			row = df.iloc[acum]
			try:
				dt_a = extract_data(json.loads(line.strip()))
				rows.append({
        			"Word_DB":row['word'],
					"AoA_DB":row["averageAoA"],
					"Word":clean_text(dt_a['Word']),
					"AoA":clean_text(dt_a["AoA"]),
				})
			except:
				rows.append({
					"Word_DB":row['word'],
					"AoA_DB":row["averageAoA"],
				})
				print(f"file_num {i}\nline_num {acum}\nline {line}")					
				errors.append({
						#"File_Num":i,
     	    			"Line_Num":acum,
						#"Line":line
					})
			acum += 1

	dt_rows = pd.DataFrame(rows)

	pearson_corr = dt_rows[['AoA_DB', 'AoA']].corr('pearson')
	print(f"Pearson_Corr: {pearson_corr["AoA_DB"]["AoA"]}")
	
	spearman_corr = dt_rows[['AoA_DB', 'AoA']].corr('spearman')
	print(f"Spearman_Corr: {spearman_corr["AoA_DB"]["AoA"]}")
		    
	return dt_rows, pd.DataFrame(errors)

file_name = out_file("FinalResults.xlsx")
clean_dtset,errors_dtset = clean(file_array)


with pd.ExcelWriter(file_name) as writer:
	clean_dtset.to_excel(writer, sheet_name='Results',index=False)
	errors_dtset.to_excel(writer, sheet_name='Errors',index=False)

Pearson_Corr: 0.7276691992436427
Spearman_Corr: 0.787075570973569
